In [ ]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

from skimage.transform import estimate_transform

In [ ]:
from enum import Enum

class PolygonType(Enum):
    CLOSED = 'closed'
    OPEN = 'open'
    TEXTURE = 'textured'
    TEXTURE_WITH_CONTOUR = 'texture with contour'
    DIRECTION = 'directionality'

In [ ]:
# mapping from old ICH-set to new set
def mapping(x):
    if x <= 153:
        return (x-100)*2+197
    elif x <= 165:
        return (x-100)*2+197-1
    else:
        return (x-100)*2+197-2

In [ ]:
stack = 'MD589'

for section in range(1, 500):

    try:
        dm = DataManager(stack=stack, section=section, load_mask=False,
                        labeling_dir='/home/yuncong/CSHL_data_labelings/')
        user, ts, _, res  = dm.load_proposal_review_result('yuncong', 'latest', 'consolidated')
    except:
        continue
    
    sys.stderr.write('%d, %d\n' % (section, mapping(section)))
    
    new_res = []
    for pol in res:
        new_pol = pol.copy()
        if len(pol['vertices']) == 0:
            continue
        new_pol['vertices'] = tf(pol['vertices'])
        new_pol['labelPos'] = tf(pol['labelPos'])
        new_res.append(new_pol)

    out_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_labelings_originalLossless/' + stack + '/' + '%04d'%mapping(section)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    pickle.dump(new_res, 
                open(out_dir + '/' + stack + '_' + '%04d'%mapping(section) + '_' + user + '_' + ts + '_consolidated.pkl', 'w'))

In [ ]:
stack = 'MD589'

for section in range(93, 368+1):

    try:
        dm = DataManager(stack=stack, section=section, load_mask=False, 
                         labeling_dir='/home/yuncong/CSHL_data_labelings_originalLossless/')
        user, ts, _, results = dm.load_proposal_review_result('yuncong', 'latest', 'consolidated')
    except:
        continue
    
    print section

    T = mapping_originalLosslessToLosslessAlignedCropped[section]
    
    new_res = []
    for pol in results:
        new_pol = pol.copy()
        if len(pol['vertices']) == 0:
            continue
        
        vs = pol['vertices']
        new_pol['vertices'] = np.dot(T, np.vstack([vs.T, np.ones((1, vs.shape[0]))]))[:2].T.astype(np.int) 
        
        vs = pol['labelPos']
        new_pol['labelPos'] = np.dot(T, np.vstack([vs.T, np.ones((1, vs.shape[0]))]))[:2].T.astype(np.int)
        
        new_res.append(new_pol)

    out_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_labelings_losslessAlignCropped/' + stack + '/' + '%04d'%mapping(section)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    pickle.dump(new_res, 
                open(out_dir + '/' + stack + '_' + '%04d'%mapping(section) + '_' + user + '_' + ts + '_consolidated.pkl', 'w'))

In [ ]:
# 197 thumbnail original new
# X1 = np.array([[972,400],
#      [655,433],
#      [813,375],
#      [743,280],
#      [831,318],
#      [989,358],
#      [897,173],
#      [837,175],
#      [970,510]])*32

# 197 lossless original new
X1 = np.array([[30921, 15414],
     [20733, 15904],
     [25821, 14339],
     [23780, 11170],
     [26512, 12577],
     [31470, 14090],
     [28864, 7992],
     [26910, 8121],
     [30677, 18911]])

# 100 lossless aligned_cropped old, with label
X2 = np.array([[10450,8558],
     [286,9548],
     [5280,7731],
     [3069,4677],
     [5883,5941],
     [10929,7200],
     [8016,1256],
     [6056,1468],
     [10359,12058]])

In [ ]:
tf = estimate_transform('similarity', X2, X1)
tf(X2)

In [ ]:
transforms = pickle.load(open(os.environ['DATA_DIR']+'/MD589_finalTransfParams.pkl', 'r'))

mapping_originalLosslessToLosslessAlignedCropped = dict()

for i in range(93, 368+1):
    T = np.linalg.inv(transforms[i])
    T[:2, 2] = (T[:2, 2] - [643, 145]) * 32
    mapping_originalLosslessToLosslessAlignedCropped[i] = T

pickle.dump(mapping_originalLosslessToLosslessAlignedCropped, 
            open(os.environ['DATA_DIR'] + '/MD589_mappingFromOriginalLosslessToLosslessAlignedCropped.pkl', 'w'))

In [ ]:
mapping_originalLosslessToThumbnailAlignedCropped = dict()

for i in range(93, 368+1):
    T = np.linalg.inv(transforms[i])
    T[:2, 2] = T[:2, 2] - [643, 145]
    mapping_originalLosslessToThumbnailAlignedCropped[i] = T

In [ ]:
dm = DataManager(stack='MD589', section=197, load_mask=False, labeling_dir='/home/yuncong/CSHL_data_labelings_originalLossless/')
user, ts, _, res2 = dm.load_proposal_review_result('yuncong', 'latest', 'consolidated')

img = imread(os.environ['DATA_DIR'] + '/MD589_thumbnail_aligned_cropped/MD589_0197_thumbnail_aligned_cropped.tif')

vs = [i for i in res2 if i['label'] == 'Pn'][0]['vertices'] / 32.

T = mapping_originalLosslessToThumbnailAlignedCropped[197]
vs = np.dot(T, np.vstack([vs.T, np.ones((1, vs.shape[0]))]))[:2].T

viz = img_as_ubyte(img)

for x,y in vs:
    cv2.circle(viz, (int(x), int(y)), 3, (255,0,0))
    
plt.figure(figsize=(20,20));
plt.imshow(viz);

In [ ]:
img =imread(os.environ['DATA_DIR'] + '/MD589_thumbnail_aligned/MD589_0197_thumbnail_aligned.tif')

In [ ]:
vs = [i for i in new_res if i['label'] == 'Pn'][0]['vertices']

In [ ]:
viz = img_as_ubyte(img)

for x,y in vs:
    cv2.circle(viz, (int(x), int(y)), 3, (255,0,0))

In [ ]:
plt.figure(figsize=(20,20));
plt.imshow(viz);